<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/support_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install clarifai

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from clarifai.rest import ClarifaiApp
from clarifai.rest import ApiError
import time
import json
import csv
import random
import pandas as pd
import requests
from lxml import html
import shutil
import os
import json


# AUTHENTICATE IN GOOGLE DRIVE
def authenticate():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive
drive = authenticate()


def create_folder_in_drive(gdrive, folder_name, parent_folder_id):
  folder_metadata = {'title': folder_name,'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [{"kind": "drive#fileLink", "id": parent_folder_id}]
                    }
  folder = gdrive.CreateFile(folder_metadata)
  folder.Upload()
  print(folder)
  # Return folder informations
  print('title: %s, id: %s' % (folder['title'], folder['id']))
  return folder['id']


def drop_unnamed(df):
  cols = [c for c in df.columns if c.lower()[:7] != 'unnamed']
  return df[cols]


def upload_file(filename, folder_id):
  drive = authenticate()
  fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
  drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
  # Check if file already exists in Google Drive (prevents duplicates)
  for file in fileList:
      if file['title'] == filename:  # The file already exists, then overwrite it
          fileID = file['id']
          drive_file = drive.CreateFile({'id': fileID, 'title': filename, 'parents': [{'id': folder_id}]})
  # Upload user picture on Google Drive
  drive_file.SetContentFile(filename)  # path of local file content
  drive_file.Upload()  # Upload the file.

In [ ]:
################################################################################
###################### DOWNLOAD USER PHOTOS ####################################

def downloadUser(business_id, user_id, photo_folder, counter):
    authenticate()
    
    filename = user_id + '.jpg'
    url = 'https://www.yelp.com/user_details?userid=' + user_id
    folder_id = photo_folder

    # CHECK DUPLICATE
    fileList = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
    exists = False
    # Check if file already exists in Google Drive (prevents duplicates)
    for file in fileList:
        if file['title'] == filename:  # The file already exists
            exists = True
    
    if exists:
      return True
    else:
      # Find user picture from web page
      page_content = requests.get(url)
      print(page_content)
      if not page_content.ok:
        return False
      tree = html.fromstring(page_content.content)
      if len(tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')) == 0:
        print('PICTURE NOT FOUND')
        print(user_id)
        return False
      else: # Picture found
        image_url = tree.xpath('//*[@id="wrap"]/div[2]/div[1]/div/div[2]/div[1]/div/div/div/a/img/@src')[0]

        drive_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
      
        response = requests.get(image_url, stream=True)

        # Create a local copy of user picture
        with open(filename, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response

        # Upload user picture on Google Drive
        drive_file.SetContentFile(filename)
        drive_file.Upload()
        if os.path.exists(filename):
            os.remove(filename)
        else:
            print("The file does not exist")
        return True


def scarica_tutti_utenti(business_id, df_utenti, photo_folder_id, startfrom):
  df_utenti = df_utenti[startfrom:]
  cnt = startfrom
  for a, user in df_utenti.iterrows():
    user_id = user['user_id']
    completed = downloadUser(business_id, user_id, photo_folder_id, cnt)
    if not completed:
      print('ULTIMO SCARICATO:', cnt)
      return False
    print(cnt)
    cnt = cnt + 1


# DOWNLOAD restaurants.txt FROM DRIVE
restaurants_dataset_id = '1BqMsph8flQMGmcE_WxYV0vSC_uqi-nge'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': restaurants_dataset_id})
download.GetContentFile('restaurants.txt')
with open('restaurants.txt') as f:
    content = f.readlines()
restaurants_id_list = [x.strip() for x in content] 

print(restaurants_id_list)


numrest = 18
id_lista = restaurants_id_list[(numrest-10):(numrest-9)]
starting_from_photo = 2299


download = drive.CreateFile({'id': '1MIWS9uI9GKxsgAlzB0wXFBiutE025u2c'}) # id file gtree.csv
download.GetContentFile('gtree.csv')
gtree = pd.read_csv('gtree.csv')

for id in id_lista:
  print(id)
  user_ranking_folder = gtree.loc[gtree['business_id']==id, 'gfolder_rankings'].tolist()[0]
  ranking_file = drive.ListFile({'q': "'" + user_ranking_folder + "' in parents and trashed=false"}).GetList()[0]  
  download = drive.CreateFile({'id': ranking_file['id']})
  download.GetContentFile('ranking_' + id + '.csv')
  ranking = pd.read_csv('ranking_' + id + '.csv')
  if starting_from_photo == 0:
    id_cartella = create_folder_in_drive(drive, str(numrest) + ' - ' + id,
                                         '12OKc1s2VeJRv5kUS_fp9f4dFLn1LkSPG')  # folder face_data>Foto_User
    gtree.loc[gtree['business_id']==id, 'gfolder_userphoto'] = id_cartella
    gtree = gtree.drop_duplicates('business_id',keep='last').reset_index(drop=True)
    gtree = drop_unnamed(gtree)
    gtree.to_csv('gtree.csv')
    upload_file('gtree.csv', '1eUt2wyCOULW0-LdL6vyRJi1mUs_U46DD') # id folder data
  else:
    id_cartella = gtree.loc[gtree['business_id']==id, 'gfolder_userphoto'].tolist()[0]
  
  success = scarica_tutti_utenti(id, ranking[['user_id']], id_cartella, starting_from_photo)
  if not success:
    print('RISTORANTE:',id)
    print(numrest)
    break
  else:
    print(numrest)
    print('FINE DOWNLOAD RISTORANTE ' + id)
    numrest += 1


In [ ]:
################################################################################
################## DOWNLOAD MISSING USER PHOTO #################################

import json
import csv
import random
import pandas as pd

def scarica_utenti_mancanti(business_id, df_utenti, photo_folder_id):
  cnt=0
  for user_id in df_utenti:
    completed = downloadUser(business_id, user_id, photo_folder_id, cnt)
    if not completed:
      print('ULTIMO NON SCARICATO:', user_id)
      return False
    print(cnt)
    cnt = cnt + 1
  return True


# DOWNLOAD restaurants.txt FROM DRIVE
restaurants_dataset_id = '1BqMsph8flQMGmcE_WxYV0vSC_uqi-nge'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': restaurants_dataset_id})
download.GetContentFile('restaurants.txt')
with open('restaurants.txt') as f:
    content = f.readlines()
restaurants_id_list = [x.strip() for x in content] 

print(restaurants_id_list)

numrest = 74
while numrest<75:

  id = restaurants_id_list[(numrest-10):(numrest-9)][0]
  download = drive.CreateFile({'id': '1MIWS9uI9GKxsgAlzB0wXFBiutE025u2c'}) # id file gtree.csv
  download.GetContentFile('gtree.csv')
  gtree = pd.read_csv('gtree.csv')

  print(id)

  user_photo_folder = gtree.loc[gtree['business_id']==id, 'gfolder_userphoto'].tolist()[0]
  photo_list = drive.ListFile({'q': "'" + user_photo_folder + "' in parents and trashed=false"}).GetList() 
  user_id_list2 = [photo['title'][0:22] for photo in photo_list]
  print(user_id_list2)

  user_ranking_folder = gtree.loc[gtree['business_id']==id, 'gfolder_rankings'].tolist()[0]
  ranking_file = drive.ListFile({'q': "'" + user_ranking_folder + "' in parents and trashed=false"}).GetList()[0]  
  download = drive.CreateFile({'id': ranking_file['id']})
  download.GetContentFile('ranking_' + id + '.csv')
  ranking = pd.read_csv('ranking_' + id + '.csv')
  user_id_list = ranking['user_id'].values.tolist()
  print(user_id_list)

  difference = (list(set(user_id_list) - set(user_id_list2)))

  print('Utenti mancanti:', len(difference))
  print(difference)
  if difference:
    success = scarica_utenti_mancanti(id, difference, user_photo_folder)
  else:
    success = True
  if not success:
    print('ERRORE RISTORANTE:',id)
    print(numrest)
    break
  else:
    print(numrest)
    print('FINE DOWNLOAD RISTORANTE ' + id)
    numrest += 1


In [ ]:
################################################################################
################ ELABORATE USER PHOTOS WITH CLARIFAI ###########################

def detect_clarifai(id, url_photo):
  app = ClarifaiApp(api_key='06ff3bafaf9b4bc6b0f3f610c0e204cf')
  model = app.models.get(model_id="c0c0ac362b03416da06ab3fa36fb58e3")
  response = model.predict_by_url(url = '%s' % url_photo)
  
  df_result = pd.DataFrame(columns=['user_id', 'age', 'gender', 'ethnicity'])

  if response['outputs'][0]['data'] != {}:
    x = 0
    while (x < len(response['outputs'][0]['data']['regions'])):
      age = response['outputs'][0]['data']['regions'][x]['data']['concepts'][0]['name']
      gender = response['outputs'][0]['data']['regions'][x]['data']['concepts'][20]['name']
      ethnicity = response['outputs'][0]['data']['regions'][x]['data']['concepts'][22]['name']

      new_row = {'user_id':id,
                 'age':age,
                 'gender':gender,
                 'ethnicity':ethnicity}
      df_result = df_result.append(new_row, ignore_index=True)
      x = x + 1
  else:
    new_row = {'user_id':id,
              'age':np.NaN,
              'gender':np.NaN,
              'ethnicity':np.NaN}
    df_result = df_result.append(new_row, ignore_index=True)
  return df_result


# LEGGO GTREE
download = drive.CreateFile({'id': '1MIWS9uI9GKxsgAlzB0wXFBiutE025u2c'}) # id file gtree.csv
download.GetContentFile('gtree.csv')
gtree = pd.read_csv('gtree.csv')
gtree = drop_unnamed(gtree)
id_clarifai_folder = '1wfZ0d3cTufosLRhPKkShf7kQbJlUbBvH' # folder data > Clarifai

# DOWNLOAD restaurants.txt FROM DRIVE
restaurants_dataset_id = '1BqMsph8flQMGmcE_WxYV0vSC_uqi-nge'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': restaurants_dataset_id})
download.GetContentFile('restaurants.txt')
with open('restaurants.txt') as f:
    content = f.readlines()
restaurants_id_list = [x.strip() for x in content] 

print(restaurants_id_list)

numrest = 70
id_restaurant = restaurants_id_list[(numrest-10):(numrest-9)][0]
startfrom = 2296

id_photo_folder = gtree.loc[gtree['business_id'] == id_restaurant, 'gfolder_userphoto'].tolist()[0]
print(id_restaurant)
print(id_photo_folder)

if startfrom == 0:
  df_clarifai = pd.DataFrame(columns=['user_id', 'age', 'gender', 'ethnicity'])
  df_clarifai.to_csv(str(numrest) + ' - ' + id_restaurant + '.csv')
  id_new_file = upload_file(str(numrest) + ' - ' + id_restaurant + '.csv', id_clarifai_folder)
  gtree.loc[gtree['business_id']==id_restaurant, 'gfolder_clarifai'] = id_new_file
  gtree.to_csv('gtree.csv')
  upload_file('gtree.csv', '1eUt2wyCOULW0-LdL6vyRJi1mUs_U46DD')
else:
  id_clarifai_file = gtree.loc[gtree['business_id']==id_restaurant, 'gfolder_clarifai'].tolist()[0]
  download = drive.CreateFile({'id': id_clarifai_file})
  download.GetContentFile(str(numrest) + ' - ' + id_restaurant + '.csv')
  df_clarifai = pd.read_csv(str(numrest) + ' - ' + id_restaurant + '.csv')

file_list = drive.ListFile({'q': "'" + id_photo_folder + "' in parents and trashed=false"}).GetList()    #cartella da cui prendo le foto

count = 0

for photo in file_list[startfrom:]:
  user_id = photo['title'][0:22] 
  count = count + 1
  if user_id not in df_clarifai['user_id'].values:
    minidf_result = detect_clarifai(user_id, photo['thumbnailLink'])
    df_clarifai = df_clarifai.append(minidf_result, ignore_index=True)
    df_clarifai = drop_unnamed(df_clarifai)
    if count == 100:
      count = 0
      df_clarifai.to_csv(str(numrest) + ' - ' + id_restaurant + '.csv')
      id_new_file = upload_file(str(numrest) + ' - ' + id_restaurant + '.csv', id_clarifai_folder)
      print('csv caricato: ' + id_new_file)
      
    print(startfrom)
    startfrom = startfrom + 1
  else:
    if count == 100:
      count = 0
    print(str(startfrom) + ' già fatto')
    startfrom = startfrom + 1
      
df_clarifai.to_csv(str(numrest) + ' - ' + id_restaurant + '.csv')
id_new_file = upload_file(str(numrest) + ' - ' + id_restaurant + '.csv', id_clarifai_folder)
print('csv caricato: ' + id_new_file) 

In [ ]:
################################################################################
############################### CHECK NUMBERS ##################################

download = drive.CreateFile({'id': '1MIWS9uI9GKxsgAlzB0wXFBiutE025u2c'}) # id file gtree.csv
download.GetContentFile('gtree.csv')
gtree = pd.read_csv('gtree.csv')
gtree = drop_unnamed(gtree)

# DOWNLOAD restaurants.txt FROM DRIVE
restaurants_dataset_id = '1BqMsph8flQMGmcE_WxYV0vSC_uqi-nge'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': restaurants_dataset_id})
download.GetContentFile('restaurants.txt')
with open('restaurants.txt') as f:
    content = f.readlines()
restaurants_id_list = [x.strip() for x in content] 

print(restaurants_id_list)

numero_risto = 67

while numero_risto < 70:
  print(numero_risto)
  id_risto = restaurants_id_list[(numero_risto-10):(numero_risto-9)][0]
  print(id_risto)

  ########################## RANKING ##############################
  user_ranking_folder = gtree.loc[gtree['business_id']==id_risto, 'gfolder_rankings'].tolist()[0]
  ranking_file = drive.ListFile({'q': "'" + user_ranking_folder + "' in parents and trashed=false"}).GetList()[0]  
  download = drive.CreateFile({'id': ranking_file['id']})
  download.GetContentFile('ranking_' + id_risto + '.csv')
  file_ranking = pd.read_csv('ranking_' + id_risto + '.csv')
  ranking_lista = file_ranking['user_id'].values.tolist()

  ########################## PHOTO ##############################
  user_photo_folder = gtree.loc[gtree['business_id']==id_risto, 'gfolder_userphoto'].tolist()[0]
  photo_lista = drive.ListFile({'q': "'" + user_photo_folder + "' in parents and trashed=false"}).GetList() 
  photo_lista = [photo['title'][0:22] for photo in photo_lista]
  seen = set()
  print('duplicates:')
  for x in photo_lista:
    if x not in seen:
        seen.add(x)
    else:
      print(x)

  ########################## CLARIFAI ##############################
  user_clarifai_id = gtree.loc[gtree['business_id']==id_risto, 'gfolder_clarifai'].tolist()[0]
  #print(user_clarifai_id)
  download = drive.CreateFile({'id': user_clarifai_id})
  download.GetContentFile('clarifai_' + id_risto + '.csv')
  file_clarifai = pd.read_csv('clarifai_' + id_risto + '.csv')

  print('tot ranking = tot foto = tot clarifai')
  tot_ranking = len(file_ranking.index)
  tot_foto = len(photo_lista)
  tot_clarifai = len(file_clarifai.drop_duplicates(subset=('user_id')).reset_index().index)

  print(str(tot_ranking) + ' = ' + str(tot_foto) + ' = ' + str(tot_clarifai))
  if tot_ranking!=tot_foto or tot_ranking!=tot_clarifai or tot_foto!=tot_clarifai:
    for unid in ranking_lista:
      if unid not in photo_lista:
        print(unid) 
  numero_risto += 1


In [ ]:
################################################################################
#################### MODIFY SOME VALUES IN GTREE ###############################

download = drive.CreateFile({'id': '1MIWS9uI9GKxsgAlzB0wXFBiutE025u2c'}) # id file gtree.csv
download.GetContentFile('gtree.csv')
df = pd.read_csv('gtree.csv')
df = drop_unnamed(df)

#BE CAREFUL AT THE INDEX
indice = 20  # 20 IN GTREE, SO 30
df.loc[indice, 'gfolder_id'] = '1GqIm0fyUQRgk8Q_rtk84ukvcYKOnDUfE'
df.loc[indice, 'gfolder_rankings'] = '1X_MPVvtJe12ipU-vjNgjz45nhWVpt3G3'
df.loc[indice, 'gfolder_groups'] = '1JbskepR_3F2a1xlxx4wOOHUbOocML_Ih'
df.loc[indice, 'gfolder_userphoto'] = '1snUVguWKYR7VzhyWrqYNCRw85tuO5ljw'

gtree = drop_unnamed(df)
gtree.to_csv('gtree.csv')
upload_file('gtree.csv', '1eUt2wyCOULW0-LdL6vyRJi1mUs_U46DD') # id folder data

In [ ]:
################################################################################
######################## INSERT ONE ROW IN GTREE ###############################

def Insert_row(row_number, df, row_value):
    start_upper = 0
    end_upper = row_number
    start_lower = row_number
    end_lower = df.shape[0]
    upper_half = [*range(start_upper, end_upper, 1)]
    lower_half = [*range(start_lower, end_lower, 1)]
    lower_half = [x.__add__(1) for x in lower_half]
    index_ = upper_half + lower_half
    df.index = index_
    df.loc[row_number] = row_value
    df = df.sort_index()
    return df 

download = drive.CreateFile({'id': '1MIWS9uI9GKxsgAlzB0wXFBiutE025u2c'}) # id file gtree.csv
download.GetContentFile('gtree.csv')
df = pd.read_csv('gtree.csv')
print(df)
df = drop_unnamed(df)
 
row_number = 66
row_value = ['fL-b760btOaGa85OJ9ut3w', '1DhrpTWJygfaPrYkHNH1xuz-HxWZBZe__',
             '1Vm5kyWMQQCSMBLSGpDwVfHIR4bp5-JIt', '19bUEYHBUGKxt7DjStk0qJIlqq4NKslw8',
             '','','','','']
  
if row_number > df.index.max()+1: 
  print("Invalid row_number") 
else:
  df = Insert_row(row_number, df, row_value) 
  print(df[row_number-1:])

gtree = drop_unnamed(df)
gtree.to_csv('gtree.csv')
upload_file('gtree.csv', '1eUt2wyCOULW0-LdL6vyRJi1mUs_U46DD') # id folder data